<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/define_onnx_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install onnx

In [15]:
# Written by refering below
# https://leimao.github.io/blog/ONNX-Python-API/

import numpy as np
import onnx


def create_initializer_tensor(
        name: str,
        tensor_array: np.ndarray,
        data_type: onnx.TensorProto = onnx.TensorProto.FLOAT
) -> onnx.TensorProto:

    # (TensorProto)
    initializer_tensor = onnx.helper.make_tensor(
        name=name,
        data_type=data_type,
        dims=tensor_array.shape,
        vals=tensor_array.flatten().tolist())

    return initializer_tensor




# Create a dummy convolutional neural network.

# IO tensors (ValueInfoProto).
model_input_name = "X"
X = onnx.helper.make_tensor_value_info(model_input_name,
                                        onnx.TensorProto.FLOAT,
                                        [1, 64, 256, 512])
model_output_name = "Y"
Y = onnx.helper.make_tensor_value_info(model_output_name,
                                        onnx.TensorProto.FLOAT,
                                        [1, 64, 254, 510])
                                        #[None, 64, 256, 512])
                                        #[None, 64, 256, 512])

kernel_shape = (3, 3)
#pads = (1, 1, 1, 1)


maxpool_node = onnx.helper.make_node(
    name="maxpool",  # Name is optional.
    op_type="MaxPool",
    inputs=[model_input_name],
    outputs=[model_output_name],
    kernel_shape = kernel_shape,
    #pads = pads,
    )


# Create the graph (GraphProto)
graph_def = onnx.helper.make_graph(
    nodes=[maxpool_node],
    name="MaxPoolTest",
    inputs=[X],  # Graph input
    outputs=[Y],  # Graph output
    initializer=[],
)

# Create the model (ModelProto)
model_def = onnx.helper.make_model(graph_def, producer_name="onnx-example")
model_def.opset_import[0].version = 11

model_def = onnx.shape_inference.infer_shapes(model_def)

onnx.checker.check_model(model_def)

onnx.save(model_def, "maxpool.onnx")

